# 🎗️ Breast Cancer Detection using PyTorch (nn.Module)

This notebook implements a **Binary Classification Model** using **Logistic Regression** structured with `torch.nn.Module`, following industry-standard practices.

### 🚀 Complete Pipeline:
1. **Import Libraries**: Load necessary Python libraries (`torch`, `nn`, `optim`).
2. **Load Dataset**: Fetch the Breast Cancer dataset from a remote URL.
3. **Data Cleaning**: Remove unused columns like `id` and `Unnamed: 32`.
4. **Train-Test Split**: Divide data into training (80%) and testing (20%) sets.
5. **Feature Scaling**: Normalize features using `StandardScaler` for better convergence.
6. **Label Encoding**: Convert categorical labels ('M', 'B') into numerical values (1, 0).
7. **Convert to PyTorch Tensors**: Prepare data for PyTorch training.
8. **Define Model**: Implement a custom class inheriting from `nn.Module`.
9. **Define Loss & Optimizer**: Use built-in `nn.BCELoss` and `torch.optim.SGD`.
10. **Training Loop**: Train the model using the standard PyTorch workflow.
11. **Evaluation**: Test the model's accuracy on unseen data.

## 1️⃣ Import Required Libraries

We import:
- `torch`: The core PyTorch library.
- `torch.nn`: For building neural network layers (`nn.Linear`, `nn.Sigmoid`).
- `torch.optim`: For optimization algorithms (`SGD`).
- `numpy`, `pandas`: For data manipulation.
- `sklearn`: For preprocessing and splitting.

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

print("Libraries imported successfully!")

## 2️⃣ Load Dataset

We load the dataset directly from the raw GitHub URL.
The dataset contains features computed from a digitized image of a fine needle aspirate (FNA) of a breast mass.

In [ ]:
url = "https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv"
df = pd.read_csv(url)

print("Dataset Shape:", df.shape)
df.head()

## 3️⃣ Data Cleaning

We inspect the dataset and remove columns that are not useful for training, specifically:
- `id`: Unique identifier for each sample.
- `Unnamed: 32`: An empty column often created during CSV reading due to trailing commas.

In [ ]:
# Remove unnecessary columns
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

print("Shape after dropping columns:", df.shape)

## 4️⃣ Train-Test Split

We separate the features (`X`) from the target variable (`y`).
- `X`: All columns except diagnosis.
- `y`: The diagnosis column ('M' or 'B').

Then, we split the data into training (80%) and testing (20%) sets.

In [ ]:
X = df.iloc[:, 1:]      # All features (excluding diagnosis at index 0)
y = df.iloc[:, 0]       # diagnosis column

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)

## 5️⃣ Feature Scaling

Standardization calculates the mean and standard deviation for each feature and scales the data so that the mean is 0 and standard deviation is 1.
This is crucial for neural networks to converge faster.

In [ ]:
scaler = StandardScaler()

# Fit on training set only to avoid data leakage
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("First 3 rows of Scaled Training Data:\n", X_train[:3])

## 6️⃣ Label Encoding

The target variable has categorical values:
- **M** (Malignant) -> Encoded as **1**
- **B** (Benign) -> Encoded as **0**

In [ ]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

print("Encoded Labels Example (First 10):", y_train[:10])

## 7️⃣ Convert to PyTorch Tensors

PyTorch works with Tensors. We convert our numpy arrays to PyTorch tensors and ensure they are of type `float`.
We also reshape the target labels to be column vectors `(N, 1)`.

In [ ]:
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor  = torch.from_numpy(X_test).float()

# Reshape labels to (N, 1)
y_train_tensor = torch.from_numpy(y_train).float().view(-1, 1)
y_test_tensor  = torch.from_numpy(y_test).float().view(-1, 1)

print("X_train_tensor shape:", X_train_tensor.shape)
print("y_train_tensor shape:", y_train_tensor.shape)

## 8️⃣ Define Neural Network Model (`nn.Module`)

We define a simple Logistic Regression model class inheriting from `nn.Module`.

**Architecture**:
- **Linear Layer**: Maps input features to a single output value.
- **Sigmoid Activation**: squashes the output between 0 and 1 (probability).

**Note**: We do NOT define the loss function inside the class.

In [ ]:
class MySimpleNN(nn.Module):

    def __init__(self, num_features):
        super().__init__()

        # Linear Layer
        self.linear = nn.Linear(num_features, 1)
        
        # Sigmoid activation
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        # Linear transformation
        out = self.linear(features)
        
        # Apply sigmoid
        out = self.sigmoid(out)
        
        return out

## 9️⃣ Training Pipeline Setup

We use specific hyperparameters and built-in Torch tools:
- **Loss Function**: `nn.BCELoss()` - Binary Cross Entropy Loss.
- **Optimizer**: `torch.optim.SGD` - Stochastic Gradient Descent.
- **Learning Rate**: 0.1
- **Epochs**: 25

In [ ]:
# Hyperparameters
learning_rate = 0.1
epochs = 25

# Create Model Instance
model = MySimpleNN(X_train_tensor.shape[1])

# Define Loss Function (Built-in)
loss_function = nn.BCELoss()

# Define Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

print("Model, Loss, and Optimizer initialized!")

## 🔟 Training Loop

The standard PyTorch training loop consists of 5 main steps:
1. **Forward Pass**: Compute prediction `y_pred` from input `X`.
2. **Calculate Loss**: Compare `y_pred` with true labels `y`.
3. **Zero Gradients**: Clear old gradients from the previous step.
4. **Backward Pass**: Compute gradients for all parameters (`loss.backward()`).
5. **Optimizer Step**: Update model parameters (`optimizer.step()`).

In [ ]:
print("Starting Training...\n")

for epoch in range(epochs):

    # ------------------
    # Forward pass
    # ------------------
    y_pred = model(X_train_tensor)

    # ------------------
    # Loss calculate
    # ------------------
    loss = loss_function(y_pred, y_train_tensor.view(-1, 1))

    # ------------------
    # Clear gradients
    # ------------------
    optimizer.zero_grad()

    # ------------------
    # Backward pass
    # ------------------
    loss.backward()

    # ------------------
    # Parameters update
    # ------------------
    optimizer.step()

    # ------------------
    # Print loss
    # ------------------
    print(f'Epoch: {epoch + 1}, Loss: {loss.item():.4f}')

## 1️⃣1️⃣ Model Evaluation

We evaluate the trained model on the unseen Test data.
- Compute predictions on `X_test_tensor`.
- Threshold probabilities at 0.5 to convert to binary classes (0 or 1).
- Compare predicted classes with true labels (`y_test_tensor`) to calculate accuracy.

In [ ]:
with torch.no_grad():
    # Get probabilities for test set
    y_pred = model(X_test_tensor)
    
    # Convert probabilities to 0/1 classes
    y_pred_cls = (y_pred > 0.5).float()

    # Calculate accuracy
    accuracy = (y_pred_cls == y_test_tensor.view(-1, 1)).float().mean()

    print(f"Test Accuracy: {accuracy.item():.4f}")
    print(f"Test Accuracy Score: {accuracy.item() * 100:.2f}%")